In [1]:
import os
import threading
import time

import schedule

# import datetime
from mkndaq.inst.neph import NEPH
from mkndaq.utils.utils import setup_logging, load_config

cfg = {'ne300': {
                'type': 'NE300',
                'serial_number': '23-0690',
                'serial_id': 0,
                'protocol': 'acoem',
                'socket': {
                    'host': '192.168.3.149',
                    'port': 32783,
                    'timeout': 10,
                },
                'data_log': {
                    'parameters': [1000000, 2000000, 3000000, 6000000, 7000000, 8000000, 
                                   11000000, 12000000, 13000000, 14000000, 15000000, 16000000, 
                                   17000000, 18000000, 19000000, 20000000, 21000000, 26000000, 
                                   5001, 5002, 5003, 5004, 5005, 5006, 
                                   5010, 6007, 6008, 6001, 6002, 6003, 
                                   4035, 4036],
                    'wavelengths': [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
                    'angles': [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
                    'interval': 60, 
                },
                'sampling_interval': 2, # minutes. logger retrieval interval
                'reporting_interval': 10,
                'zero_span_check_interval': 1500,
                'zero_check_duration': 30,
                'span_check_duration': 0,
                'staging_zip': True,  
                'verbosity': 2,  # 0: silent, 1: medium, 2: full          
                'data_path': 'data/ne300-test',
                'staging_path': 'staging/ne300-test',
                'remote_path': 'ne300',
            },
            'reporting_interval': 10,
            # 'logs': 'mkndaq/logs',
    'logging': {
        'file': 'mkndaq.log',
        'level_console': 'DEBUG',
        'level_file': 'ERROR',
    },
    'root': '~/Documents/mkndaq',
}

# """Read config file, set up instruments, and launch data acquisition."""
# # collect and parse CLI arguments
# parser = argparse.ArgumentParser(
#     description='Data acquisition and transfer for MKN Global GAW Station.',
#     usage='python3 mkndaq.py|mkndaq.exe -c [-f]')
# parser.add_argument('-c', '--configuration', type=str,
#                     help='full path to configuration file',
#                     default='dist/mkndaq.yml', required=False)
# parser.add_argument('-f', '--fetch', type=int, default=20,
#                     help='interval in seconds to fetch and display current instrument data',
#                     required=False)
# args = parser.parse_args()
# fetch = args.fetch
# config_file = args.configuration
config_file = 'C:/Users/mkn/Documents/git/mkndaq/dist/mkndaq.yml'

# load configuation
cfg = load_config(config_file=config_file)

# setup logging
logfile = os.path.join(os.path.expanduser(str(cfg['root'])),
                        cfg['logging']['file'])
logger = setup_logging(file=logfile)
logger.error('test error logging')

ne300 = NEPH('ne300', cfg, verbosity=0)
# # Initialize NEPH (name: ne300  S/N: 23-0690)
#   Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.

2024-10-13T07:40:19, ERROR, mkndaq, test error logging
2024-10-13T07:40:19, INFO, mkndaq.mkndaq.inst.neph, [ne300] Initializing NE300 (S/N: 23-0690)
2024-10-13T07:40:19, INFO, mkndaq.mkndaq.inst.neph, get_id: {'id': 'ACOEM Aurora NE-300 Sub-Type: 0 Range: 0 Build: 158 Branch: 300'}
2024-10-13T07:40:19, INFO, mkndaq.mkndaq.inst.neph, [ne300] Instrument identified itself as '{'id': 'ACOEM Aurora NE-300 Sub-Type: 0 Range: 0 Build: 158 Branch: 300'}'.
2024-10-13T07:40:19, INFO, mkndaq.mkndaq.inst.neph, [ne300] Instrument current operation is 'ambient'.
2024-10-13T07:40:19, INFO, mkndaq.mkndaq.inst.neph, [ne300] dtm found: 2024-10-13 07:40:18 > dtm set: 2024-10-13 07:40:18.
2024-10-13T07:40:19, INFO, mkndaq.mkndaq.inst.neph, [ne300] Logging config reported by instrument: [2635000, 2525000, 2450000, 2635090, 2525090, 2450090, 5001, 5002, 5003, 5004, 5005, 5006, 5010, 26635000, 26525000, 26450000, 13525000, 15635000, 15525000, 15450000, 11635000, 11525000, 11450000, 11635090, 11525090, 114500

In [2]:
import logging

def run_threaded(job_func):
    """Set up threading and start job.

    Args:
        job_func ([type]): [description]
    """
    job_thread = threading.Thread(target=job_func)
    job_thread.start()

# limit logging from schedule
logging.getLogger('schedule').setLevel(logging.CRITICAL)

# align start with a 10' timestamp
while int(time.time()) % 10 > 0:
    time.sleep(0.1)

fetch = 40
schedule.every(fetch).seconds.do(ne300.print_ssp_bssp)
schedule.every(cfg['ne300']['sampling_interval']).minutes.at(':10').do(run_threaded, ne300._accumulate_new_data)
# schedule.every(cfg['ne300']['zero_span_check_interval']).minutes.at(':00').do(run_threaded, ne300.do_zero_span_check)
for minute in range(6):
    schedule.every().hour.at(f"{minute}0:01").do(ne300._save_and_stage_data)

while True:
    schedule.run_pending()
    time.sleep(1)

2024-10-13T07:41:10, INFO, mkndaq.mkndaq.inst.neph, [ne300] ssp|bssp (Mm-1) r: 28.0553|4.4598 g: 39.8927|6.2728 b: 50.5552|7.3606
2024-10-13T07:41:50, INFO, mkndaq.mkndaq.inst.neph, [ne300] ssp|bssp (Mm-1) r: 27.7945|4.4351 g: 39.8709|6.2399 b: 50.4512|7.3168
2024-10-13T07:42:10, INFO, mkndaq.mkndaq.inst.neph, [ne300] .accumulate_new_data
2024-10-13T07:42:10, INFO, mkndaq.mkndaq.inst.neph, [{2635000: 28.34547, 2525000: 40.13524, 2450000: 50.23344, 2635090: 4.48447, 2525090: 6.13373, 2450090: 7.33713, 5001: 28.43002, 5002: 658.96002, 5003: 25.5, 5004: 29.45001, 5005: 664.01001, 5006: 33.39, 5010: 5.00521, 26635000: 0.02532, 26525000: 0.02738, 26450000: 0.03069, 13525000: 270.0051, 15635000: 1749299.0, 15525000: 2035618.0, 15450000: 1479490.0, 11635000: 19606.86914, 11525000: 27456.0, 11450000: 26791.82031, 11635090: 12397.41992, 11525090: 15284.66992, 11450090: 13596.25977, 6007: 24.11865, 6008: 1925.31702, 6001: 18.33984, 6002: 4.95117, 6003: 20.60881, 6635000: 0.01177, 6525000: 0.0140

KeyboardInterrupt: 

In [ ]:
resp = ne300.get_values([5006])
time.sleep(.5)
print(resp)

In [ ]:
ne300.set_value(2030, 5006, verify=False, verbosity=2)
# ne300.set_value(4035, 1)

In [ ]:
ne300.get_data_log_config()

In [ ]:
res = ne300.get_id()
print(res)

In [ ]:
# mimick VI099 response
ne300.get_current_data(strict=True, verbosity=0)

In [ ]:
ne300.get_current_data(verbosity=0)

In [ ]:
ne300.get_values([1635000, 1525000], verbosity=2)

In [ ]:
# retrieve VI099 command parameters
ne300.get_values(parameters=[1, 1635000,1525000,1450000,1635090,1525090,1450090,5001,5004,5003,5002,4036,4035])

In [ ]:
ne300.get_current_data(add_params=[1001, 1002, 1003, 1004, 1005, 2001, 2002])

In [ ]:
# not implemented
ne300._accumulate_new_data()
ne300._data

In [ ]:
# not implemented
# ne300.get_all_data()

In [ ]:
ne300.get_instr_type()

In [ ]:
ne300.get_datetime()

In [ ]:
ne300.get_data_log_config()

In [ ]:
# cycle through operating states
# 0: ambient, 1: zero, 2: span
for i in [1, 2, 0]:
    print(f"current: {ne300.get_current_operation()} > set: {i} ...")
    print(f"done: {ne300.set_current_operation(state=i)} okay")
    time.sleep(3)

In [ ]:
from datetime import datetime, timezone, timedelta
end = datetime.now(timezone.utc)
start = end - timedelta(minutes=5)
print(f"start: {start}\nend  : {end}")
data = ne300.get_logged_data(start=start, end=end, verbosity=1)
# for i in range(len(data)):
    # print(data[i])

In [ ]:
# repeat packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,1]))
response = ne300._tcpip_comm(message=message)
print(response)
display(ne300._acoem_decode_logged_data(response=response))

In [ ]:
# next packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,0]))
response = ne300._tcpip_comm(message=message)
response